# Qrack and cuQuantum

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

You also need the `cuQuantum` package, just for this notebook. [`NVIDIA cuQuantum`](https://docs.nvidia.com/cuda/cuquantum/latest/) provides APIs including state vector simulation and tensor network simulation. In this notebook, we'll show you how take a random or arbitrary circuit, simplify it with the `QuantumCircuit` class from Qrack, convert it into a Qiskit circuit in proper format to load as a tensor network with `CircuitToEinsum` in `cuQuantum`, then do whatever simulation and manipulation of the tensor network you wish with `cuQuantum`.

In [1]:
# How wide of an example circuit should we generate? (In qubits)
width = 6

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack cuquantum qiskit numpy

# import os

# If simulating with Qrack, change these according to your OpenCL devices and system specifications.
# (To use just `QrackCircuit`, we don't need to consider these.)
# os.environ['QRACK_OCL_DEFAULT_DEVICE']='0'
# os.environ['QRACK_QUNITMULTI_DEVICES']='0'
# os.environ['QRACK_QPAGER_DEVICES']='0'
# os.environ['QRACK_QPAGER_DEVICES_HOST_POINTER']='0'
# os.environ['QRACK_MAX_PAGE_QB']='27'
# os.environ['QRACK_MAX_ALLOC_MB']='15872'.
# os.environ['QRACK_MAX_PAGING_QB']='30'
# os.environ['QRACK_MAX_CPU_QB']='32'
# os.environ['QRACK_QTENSORNETWORK_THRESHOLD_QB']='30'

All this next cell does is provide a function for generating random circuits, for our example that follows.

In [3]:
import math
import random

sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(circ, q):
    circ.mtrx([1, 0, 0, 1j], q)
    return 1

def x_to_z(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def y_to_z(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 2

def y_to_x(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    return 1

def z_to_x(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def z_to_y(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    circ.mtrx([1, 0, 0, 1j], q)
    return 2

def cx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 1)
    return 1

def cy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 1)
    return 1

def cz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    return 1

def acx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 0)
    return 1

def acy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 0)
    return 1

def acz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 1

def swap(circ, q1, q2):
    circ.swap(q1, q2)
    return 1

def nswap(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 3

def pswap(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    circ.swap(q1, q2)
    return 2

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 2

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    circ.mtrx([1, 0, 0, 1j], q1)
    circ.mtrx([1, 0, 0, 1j], q2)
    return 4

def iiswap(circ, q1, q2):
    circ.mtrx([1, 0, 0, -1j], q2)
    circ.mtrx([1, 0, 0, -1j], q1)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    circ.swap(q1, q2)
    return 4

def random_circuit(width, circ):
    gate_count = 0
    
    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) } 
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))

    # Don't repeat bases:
    bases = [0] * width
    directions = [0] * width
    
    for i in range(3 * width):
        # Single bit gates
        for j in range(width):
            # Reset basis, every third layer
            if i % 3 == 0:
                bases[j] = random.randint(0, 2)
                directions[j] = random.randint(0, 1)
            
            # Sequential basis switch
            gate = single_bit_gates[bases[j]][directions[j]]
            gate_count += gate(circ, j)

            # Cycle through all 3 Pauli axes, every 3 layers
            if directions[j]:
                bases[j] -= 1
                if bases[j] < 0:
                    bases[j] += 3
            else:
                bases[j] += 1
                if bases[j] > 2:
                    bases[j] -= 3
                
            # Rotate around local Z axis
            rnd = random.uniform(0, 2 * math.pi)
            circ.mtrx([1, 0, 0, math.cos(rnd) + math.sin(rnd) * 1j], j)
            gate_count += 1
            
        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                gate_count += g(circ, b1, b2)

    print("Gate count (before optimization): " + str(gate_count))

    return circ

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size.

`QrackCircuit` can be converted to-and-from Qiskit's `QuantumCircuit` class, for ease of use. You'll notice that `QrackCircuit` needs the intermediary of disk input/output to do this, in cases: because `QrackCircuit` is ultimately at "C++ level," rather than "Python level," it can't directly do things like print circuit diagrams or list its internal gates. However, with disk I/O, it can rely on Qiskit for this, while `QrackCircuit` provides a fairly powerful way of simplifying input circuits and outputting fully-variational Qiskit circuits.

In [4]:
from pyqrack import QrackCircuit

qcircuit = QrackCircuit(width)
random_circuit(width, qcircuit)

qcircuit.out_to_file('qrack_circuit.qc')

data = ''
with open('qrack_circuit.qc', 'r') as file:
    data = file.read().split()
    
print("Gate count (after optimization): " + str(data[1]))

Gate count (before optimization): 298
Gate count (after optimization): 63


If we have Qiskit and numpy installed, we can convert to a Qiskit circuit.

In [5]:
circ = QrackCircuit.file_to_qiskit_circuit('qrack_circuit.qc')
print(circ)

                                                    ┌──────────────┐»
q_0: ───────────────────────────────────────────────┤0             ├»
                                    ┌──────────────┐│              │»
q_1: ───────────────────────────────┤0             ├┤              ├»
                    ┌──────────────┐│              ││  Multiplexer │»
q_2: ───────────────┤0             ├┤              ├┤              ├»
     ┌─────────────┐│              ││  Multiplexer ││              │»
q_3: ┤ Multiplexer ├┤              ├┤              ├┤1             ├»
     ├─────────────┤│  Multiplexer ││              │├─────────────┬┘»
q_4: ┤ Multiplexer ├┤              ├┤1             ├┤ Multiplexer ├─»
     ├─────────────┤│              │└──────────────┘└─────────────┘ »
q_5: ┤ Multiplexer ├┤1             ├────────────────────────────────»
     └─────────────┘└──────────────┘                                »
«                                                                    »
«q_0: ─────────────

This form in which `QrackCircuit` outputs a Qiskit `QuantumCircuit` is **immediately** of a form that that is entirely compatible with the `CircuitToEinsum` class of (Python) `cuQuantum`, which converts a quantum circuit representation to "Einstein notation" that is natively understood by `cuTensorNet`.

In [6]:
from cuquantum import contract, CircuitToEinsum

converter = CircuitToEinsum(circ, backend='cupy')
expression, operands = converter.amplitude('0' * width)
print(contract(expression, *operands))

(0.04065213631788436+0.09303515895304368j)


**(Happy Qracking! You rock!)**